In [1]:
import pandas as pd
import numpy as np
from glob import glob
import gcsfs

### Find datasets which have a new version since a prior date
 - Specify the catalog date to compare to current catalog
 - Specify catalog type ('-noQC' or '') 

In [2]:
fs = gcsfs.GCSFileSystem(token='anon',access='read_only',cache_timeout=-1)

date = '20210118'
cat_type = '-noQC'

# find closest catalog to specified date:
files = fs.glob(f'gs://cmip6/old_catalogs/pangeo-cmip6-*{cat_type}.csv.gz')
dates = [int(s.split('cmip6-')[-1].split(cat_type)[0]) for s in files]
prior_date = str(min(dates, key=lambda x:abs(x-int(date))))
prior_date

'20210118'

In [3]:
# Read the catalogs
df_old = pd.read_csv(f'https://cmip6.storage.googleapis.com/old_catalogs/pangeo-cmip6-{prior_date}{cat_type}.csv.gz', dtype='unicode')
df = pd.read_csv(f'https://cmip6.storage.googleapis.com/cmip6-zarr-consolidated-stores{cat_type}.csv', dtype='unicode')

In [4]:
dfn = pd.merge(df_old, df[['zstore','version']], on='zstore', how='left', sort=False, suffixes=('_old', '_new'))
df_replaced = dfn[dfn.version_old != dfn.version_new]
df_replaced = df_replaced[~(df_replaced.version_new.isnull())]

In [5]:
df_replaced[['zstore', 'version_old','version_new']]

,zstore,version_old,version_new
45317,gs://cmip6/CMIP/E3SM-Project/E3SM-1-1-ECA/hist...,20200127,20200623
46437,gs://cmip6/CMIP/EC-Earth-Consortium/EC-Earth3-...,20190605,20200225
106160,gs://cmip6/CMIP/NCC/NorCPM1/historical/r1i1p1f...,20190914,20200724
116537,gs://cmip6/CMIP/THU/CIESM/historical/r1i1p1f1/...,20191228,20200417
297817,gs://cmip6/ScenarioMIP/CAS/FGOALS-g3/ssp126/r1...,20190818,20200927
341957,gs://cmip6/ScenarioMIP/EC-Earth-Consortium/EC-...,20190629,20200225
343887,gs://cmip6/ScenarioMIP/EC-Earth-Consortium/EC-...,20190926,20200310
344680,gs://cmip6/ScenarioMIP/EC-Earth-Consortium/EC-...,20190927,20200310
